In [2]:
import astropy.units as units
import astropy.constants as constants
import matplotlib.pyplot as plt
import sympy as sym
import numpy as np 
import pandas as pd
import plotly.express as px
import plotly.figure_factory as ff
import plotly.graph_objects as go
import requests
import re
import subprocess
import urllib.request
from sympy.abc import *
from bs4 import BeautifulSoup
import csv
cwd = subprocess.os.getcwd()

In [2]:
def half_life_to_lambda(half_life):
    '''
    This function takes the e-folding times of the entire decay chain.
    Returns the half-life of the nuclide
    '''
    return half_life / np.log(2)

nuclide_df = pd.read_csv(cwd + '\\NuclideData.csv').iloc[:,1:]
half_lives = nuclide_df['Half life (years)'].to_numpy() 
nuclide_df["e Folding Time (seconds)"] = half_life_to_lambda(half_lives * units.year.to(units.s))
nuclide_df

,Beta-decay fraction,Average beta decay energy,Beta-decay energy (keV),Half life (years),Isotope,e Folding Time (seconds)
0,0.0,5.69,4065.0,7.288260e-07,237Ac,33.181986
1,0.0,5.69,-14187.0,3.168809e-08,130Sm,1.442695
2,0.0,5.69,-2474.0,6.337618e-07,287Nh,28.853901
3,1.0,5.69,14464.0,4.309580e-09,134In,0.196207
4,1.0,5.69,8610.0,1.679469e-09,154Ba,0.076463
...,...,...,...,...,...,...
3187,0.0,5.69,5780.0,1.394276e-08,176Dy,0.634786
3188,0.0,1067.00,-428.9806,6.445357e-06,140Pr,293.444171
3189,0.0,5.69,-8779.9397,3.263873e-07,152Er,14.859759
3190,0.0,5.69,-2824.0194,5.970036e-06,164Lu,271.803746


In [3]:
element_symbols = ['H', 'He', 'Li', 'Be', 'B', 'C', 'N', 'O', 
'F', 'Ne', 'Na', 'Mg', 'Al', 'Si', 'P', 'S', 'Cl', 'Ar', 'K', 
'Ca', 'Sc', 'Ti', 'V', 'Cr', 'Mn', 'Fe', 'Co', 'Ni', 'Cu', 
'Zn', 'Ga', 'Ge', 'As', 'Se', 'Br', 'Kr', 'Rb', 'Sr', 'Y', 
'Zr', 'Nb', 'Mo', 'Tc', 'Ru', 'Rh', 'Pd', 'Ag', 'Cd', 'In', 
'Sn', 'Sb', 'Te', 'I', 'Xe', 'Cs', 'Ba', 'La', 'Ce', 'Pr', 
'Nd', 'Pm', 'Sm', 'Eu', 'Gd', 'Tb', 'Dy', 'Ho', 'Er', 'Tm',
 'Yb', 'Lu', 'Hf', 'Ta', 'W', 'Re', 'Os', 'Ir', 'Pt', 'Au', 
 'Hg', 'Tl', 'Pb', 'Bi', 'Po', 'At', 'Rn', 'Fr', 'Ra', 'Ac', 
 'Th', 'Pa', 'U', 'Np', 'Pu', 'Am', 'Cm', 'Bk', 'Cf', 'Es', 
 'Fm', 'Md', 'No', 'Lr', 'Rf', 'Db', 'Sg', 'Bh', 'Hs', 'Mt',
  'Ds', 'Rg', 'Cn', 'Nh', 'Fl', 'Mc', 'Lv', 'Ts', 'Og', 'N/A']
len(element_symbols)
#to find the daughter nuclide we only need to increment by 1 in the element symbols.
element_symbols[element_symbols.index('Ni')+1]
#remove numbers from string
s = element_symbols
nuclide_df['Daugher Nucleus'] = [re.sub('\D+', '', n) + s[s.index(re.sub('\d+', '', n))+1] 
if re.sub('\d+', '', n) in s else 'N/A' for n in nuclide_df['Isotope'] ]

In [4]:
def get_isotope_info(isotope, info = None, isotope_column = None,
    dataset = {}, isotope_list = None, lists_to_search = []):                  
  '''
  isotope_list and list_to_search are optional arguments.
  If list_to_search is not provided, then info must be provided.
  If isotope_list is not provided, then dataset and isotope_column
  must be provided.
  '''
  if isotope_list is None:
    isotope_list = list(dataset[isotope_column])
  row = isotope_list.index(isotope)
  if len(lists_to_search) == 0:
    try:
      lists_to_search = list(dataset[info])
    except:
      print("info to search for not entered")
      return
  try: #only works if there are multiple specified lists to search 
    return [target_list[row] for target_list in lists_to_search]
  except:
    return lists_to_search[row]


In [5]:
isotope_list = list(nuclide_df['Isotope'])
lambda_list = list(nuclide_df['e Folding Time (seconds)'])
decay_energy_list = list(nuclide_df['Average beta decay energy'])
daughter_list = list(nuclide_df['Daugher Nucleus'])

In [6]:
def make_decay_chain(isotope, isotope_list, lambda_list, decay_energy_list, daughter_list):
    '''
    This function takes the isotope, e-folding times, and daughter nuclides
    and returns the decay chain.
    '''
    decay_chain = {}
    decay_chain[isotope] = get_isotope_info(isotope, isotope_list = isotope_list, 
        lists_to_search = (lambda_list, decay_energy_list, daughter_list))
    while True:
        isotope = decay_chain[isotope][2]
        try:
            decay_chain[isotope] = get_isotope_info(isotope, isotope_list = isotope_list,
                lists_to_search = (lambda_list, decay_energy_list, daughter_list))
            if(isotope == decay_chain[isotope][2]):
                return decay_chain
        except:
            return pd.DataFrame(decay_chain, index= ("e-Folding Time (seconds)", 
                                "Average beta-decay energy", "Daughter")).transpose()


In [7]:
#Quickly calculating the decay rate of the i-th generation nuclide
def formulate_decay_rate(e_folding_times):
    '''
    The e-folding times must be in a numpy array.
    Returns a formula for the decay rate of each generation. 
    '''
    exponent_array = -1 / e_folding_times
    decay_rates = len(exponent_array)  * [sym.N(0)]
    decay_rates[0] = sym.exp(t * exponent_array[0]) * exponent_array[0]
    for index, L in enumerate(exponent_array[1:]):
        decay_rates[index+1] = decay_rates[index] * (1 + sym.exp(t * L) * L) 
    return decay_rates

def eval_decay_rates(decay_rates, time_array):
    '''
    This function takes the formula for the decay rate of each generation
    and substitutes each value in the time array for t.
    Each decay_rate must be a sympy expression. 
    https://docs.sympy.org/ 
    Rewrite so that the outer loop is over the time array and the inner loop
    is over the generations. This can be done by using (suggested by copilot: 
    the np.meshgrid function) or a 2d array (my first thought)
    '''
    try: #will only evaluate if decay_rates is an array of sympy expressions
        evaluated_decay_rates = [np.array([formula.subs(t, time) for formula in decay_rates])
                                    for time in time_array]
    except:
        evaluated_decay_rates = [decay_rates.subs(t, time) for time in time_array]
    return evaluated_decay_rates


#Calculating the power density of a decay chain
def calc_power_density(decay_rates, decay_energies, initial_mass):
    '''
    Rewrite such that is sums the entire chain and NOT across time
    decay_rates must be a numpy array in moles/second
    decay_energies must be a numpy array in keV/decay
    This function takes the decay rates and energies of the decay chain.
    Returns the power density in watts/g
    '''
    power_density =  [sum(decay_rate_t * decay_energies) for decay_rate_t in decay_rates]
    '''
    At every different value for time, 
    store the sum of the product of each generation's decay rate and decay energy.
    '''
    power_density = np.array(power_density)
    #convert to W/g
    power_density *=  units.keV.to(units.J) * float(constants.N_A * units.mol) / initial_mass
    return np.abs(power_density.astype(float))


In [8]:
chain = make_decay_chain('32Si', isotope_list, lambda_list, decay_energy_list, daughter_list)
chain

,e-Folding Time (seconds),Average beta-decay energy,Daughter
32Si,7.14789e+09,68.8,32P
32P,1.77861e+06,694.9,32S


In [9]:
decay_rates = formulate_decay_rate(chain['e-Folding Time (seconds)'])
#currently, eval_decay_rates evalautes in < 1 second for 100 time points and 6 generations
time_array = np.logspace(1, 9.5, 10**2)
n_decay_rates = eval_decay_rates(decay_rates, time_array)
n_decay_rates

[array([-1.39901329266888e-10, -1.39901250609852e-10], dtype=object),
 array([-1.39901329224105e-10, -1.39901250567165e-10], dtype=object),
 array([-1.39901329171969e-10, -1.39901250515147e-10], dtype=object),
 array([-1.39901329108436e-10, -1.39901250451758e-10], dtype=object),
 array([-1.39901329031016e-10, -1.39901250374512e-10], dtype=object),
 array([-1.39901328936672e-10, -1.39901250280382e-10], dtype=object),
 array([-1.39901328821705e-10, -1.39901250165675e-10], dtype=object),
 array([-1.39901328681607e-10, -1.39901250025894e-10], dtype=object),
 array([-1.39901328510885e-10, -1.39901249855557e-10], dtype=object),
 array([-1.39901328302845e-10, -1.39901249647987e-10], dtype=object),
 array([-1.39901328049328e-10, -1.39901249395043e-10], dtype=object),
 array([-1.39901327740394e-10, -1.39901249086808e-10], dtype=object),
 array([-1.39901327363930e-10, -1.39901248711194e-10], dtype=object),
 array([-1.39901326905173e-10, -1.39901248253474e-10], dtype=object),
 array([-1.399013263

In [10]:
t_power_densities = calc_power_density(decay_rates = n_decay_rates, decay_energies = np.array(chain['Average beta-decay energy']),
        initial_mass = float(re.sub('\D+', '', chain.index[0])))
px.scatter(y=t_power_densities, x= time_array, log_x=True, log_y=True,
        labels={'x': 'Time (seconds)', 'y': 'Power Density (W/g)'}, title = 'Power Density of ' + chain.index[0] + ' Decay Chain')
#THIS IS MUCH HIGHER THAN WHAT I HAVE IN THE DECADE HALF LIVES POWER DENSITIES ON GITHUB!!!
#Double check decay formulas!!!!

In [11]:
time_array

array([1.00000000e+01, 1.21859274e+01, 1.48496826e+01, 1.80957154e+01,
       2.20513074e+01, 2.68715631e+01, 3.27454916e+01, 3.99034183e+01,
       4.86260158e+01, 5.92553098e+01, 7.22080902e+01, 8.79922544e+01,
       1.07226722e+02, 1.30665705e+02, 1.59228279e+02, 1.94034425e+02,
       2.36448941e+02, 2.88134963e+02, 3.51119173e+02, 4.27871275e+02,
       5.21400829e+02, 6.35375264e+02, 7.74263683e+02, 9.43512101e+02,
       1.14975700e+03, 1.40108553e+03, 1.70735265e+03, 2.08056754e+03,
       2.53536449e+03, 3.08957676e+03, 3.76493581e+03, 4.58792343e+03,
       5.59081018e+03, 6.81292069e+03, 8.30217568e+03, 1.01169710e+04,
       1.23284674e+04, 1.50233808e+04, 1.83073828e+04, 2.23092437e+04,
       2.71858824e+04, 3.31285189e+04, 4.03701726e+04, 4.91947992e+04,
       5.99484250e+04, 7.30527154e+04, 8.90215085e+04, 1.08480964e+05,
       1.32194115e+05, 1.61090788e+05, 1.96304065e+05, 2.39214708e+05,
       2.91505306e+05, 3.55226249e+05, 4.32876128e+05, 5.27499706e+05,
      

In [12]:
all_decay_chains = [make_decay_chain(isotope, isotope_list, lambda_list, decay_energy_list, daughter_list) 
                    for isotope in isotope_list]
time_array = np.logspace(1, 9.5, 10)
def calculate_all_power_densities(all_decay_chains, time_array, mean = True):
    '''
    This function takes the isotope, e-folding times, and daughter nuclides
    and returns power density as a function of time for each decay chain.
    The optional boolean argument mean determines whether the whole time series is returned
    or only its mean. 
    '''
    if mean:
        all_power_densities = [np.mean(calc_power_density(
                                    decay_rates = eval_decay_rates(
                                                    formulate_decay_rate(chain['e-Folding Time (seconds)']), 
                                                    time_array), 
                                    decay_energies = np.array(chain['Average beta-decay energy']),
                                    initial_mass = float(re.sub('\D+', '', chain.index[0]))
                                )) for chain in all_decay_chains]
    else:
        all_power_densities = [calc_power_density(
                                    decay_rates = eval_decay_rates(
                                                    formulate_decay_rate(chain['e-Folding Time (seconds)']), 
                                                    time_array), 
                                    decay_energies = np.array(chain['Average beta-decay energy']),
                                    initial_mass = float(re.sub('\D+', '', chain.index[0]))
                                ) for chain in all_decay_chains]
    return all_power_densities

mean_power_densities = calculate_all_power_densities(all_decay_chains[:2], time_array, mean = True)


In [13]:
3.7 * len(all_decay_chains) * units.second.to(units.h) / 2
#it will take 2 hours to run my script in jupyter. I'l convert it to a python script and run it in pypy to see if it's faster.


1.6403333333333336

In [14]:
#Import the file exported on colab
relative_path = '\MeanPowerDensitiesofDecayChains\SortedPowerDensities1yto100yMean.csv'
rho_p_1_100 = pd.read_csv(cwd + relative_path).rename(
        columns={'Unnamed: 0': 'Unsorted Index', 
        'Mean Power Density Over 100 years (watts per gram)':'Power Density (W/g)'})
rho_p_1_100 #power density by chain with 100 points in the time series from 1y to 100y.

,Unsorted Index,Parent Isotope,Power Density (W/g)
0,422,228Th,8.911231
1,1913,254Es,8.316062
2,1881,228Ra,6.341824
3,323,57Co,6.085650
4,2043,252Es,5.031617
...,...,...,...
3187,1153,155Sm,0.000000
3188,1154,51Ar,0.000000
3189,1155,111Sn,0.000000
3190,1156,14O,0.000000


In [15]:
#Evaluate the product of power density of the chain and half life of the parent
def eval_battery(half_life, power_density):
    evaluation = 0
    try:
        half_life = min(half_life, 100)
        half_life = half_life / 20
        coeff = min(half_life, power_density) ** 2
        if coeff > 0:
            evaluation = max(0, (coeff * half_life * power_density))
    except: 
        return 0
    return evaluation

parent_half_lives = [all_decay_chains[unsorted_index]['e-Folding Time (seconds)'][0] 
                        / units.year.to(units.second) for row, unsorted_index 
                in enumerate(rho_p_1_100['Unsorted Index'])]
evaluations = [eval_battery(half_life, rho_p_1_100['Power Density (W/g)'][row])
                for row, half_life in enumerate(parent_half_lives)]    
rho_p_1_100['Evaluation'] = np.array(evaluations).astype(float) / max(evaluations)
rho_p_1_100['Half Life (years)'] = np.array(parent_half_lives).astype(float)
#sort dataframe by evaluations
rho_p_1_100 = rho_p_1_100.sort_values(by = 'Evaluation', ascending = False)
rho_p_1_100

,Unsorted Index,Parent Isotope,Power Density (W/g),Evaluation,Half Life (years)
7,236,236Pu,4.124938,1.000000,1.364790e+02
19,1162,65Zn,1.091814,0.018544,9.465522e+03
29,1848,139Ce,0.485416,0.001630,7.574149e+04
30,657,88Y,0.398075,0.000899,6.789323e+03
24,1262,232U,0.646514,0.000684,1.777400e+01
...,...,...,...,...,...
1299,3008,172Tb,0.000000,0.000000,3.300713e-08
1300,3009,81Sr,0.000000,0.000000,1.442805e-04
1301,3011,228Ac,0.000000,0.000000,2.925840e-07
1302,3039,165Eu,0.000000,0.000000,1.298341e-06


In [16]:
px.scatter(rho_p_1_100 , x = 'Half Life (years)', 
            y = 'Power Density (W/g)', log_x = True,
            hover_name = 'Parent Isotope',
            color = 'Evaluation', color_continuous_scale = 'algae')
            

In [17]:
np.round(10.0005, 4)

10.0005

In [18]:
def clean_time_series(time_series):
    '''
    This function takes the time series of power densities and returns a list
    of lists of the time series converted into floats.
    '''
    split_time_series = time_series.replace('[', ' ').replace(']', ' ').replace('\n', ' ').split(' ') 
    return np.round(np.array([float(power_density)  
                    for power_density in split_time_series 
                    if len(power_density)>0]), 2)

time_series_file_path = cwd + '\MeanPowerDensitiesofDecayChains\power_density_time_series.csv'
#read the file to a numpy array
with open(time_series_file_path, newline='\n') as f:
    reader = csv.reader(f)
    time_series_unclean = list(reader)[0]
power_densities_dict = {}
power_densities_dict['Time (years)'] = np.logspace(0, 9.5, 10**3) * units.second.to(units.year)
for row, chain in enumerate(all_decay_chains):
    power_densities_dict[chain.index[0]] = clean_time_series(time_series_unclean[row])
power_densities_df = pd.DataFrame.from_dict(power_densities_dict).set_index('Time (years)')
transposed_power_densities_df = power_densities_df.transpose()
transposed_power_densities_df

Time (years),3.168809e-08,3.238960e-08,3.310664e-08,3.383955e-08,3.458869e-08,3.535441e-08,3.613708e-08,3.693708e-08,3.775479e-08,3.859061e-08,...,8.228306e+01,8.410464e+01,8.596654e+01,8.786967e+01,8.981492e+01,9.180324e+01,9.383557e+01,9.591290e+01,9.803621e+01,1.002065e+02
237Ac,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
130Sm,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
287Nh,435636.52,439796.86,443847.06,447782.55,451599.08,455292.78,458860.17,462298.10,465603.83,468774.97,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
134In,1550902.27,1551523.68,1551949.04,1552191.70,1552264.34,1552178.95,1551946.84,1551578.60,1551084.14,1550472.69,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
154Ba,0.01,0.01,0.01,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
176Dy,51446960.90,50584906.30,49718680.40,48848591.80,47974961.30,47098120.90,46218414.20,45336196.10,44451832.80,43565701.30,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
140Pr,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
152Er,6433.35,6433.22,6433.10,6432.97,6432.84,6432.71,6432.57,6432.43,6432.29,6432.15,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
164Lu,10506.59,10506.58,10506.58,10506.58,10506.57,10506.57,10506.56,10506.56,10506.56,10506.55,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [19]:
power_densities_df

,237Ac,130Sm,287Nh,134In,154Ba,155Tb,173Er,214Ac,66Co,88Kr,...,273Rg,181Yb,103Mo,92Pd,141Eu,176Dy,140Pr,152Er,164Lu,228Fr
Time (years),,,,,,,,,,,,,,,,,,,,,
3.168809e-08,0.0,0.0,435636.52,1550902.27,0.01,41450532.8,1.66,24304.35,141396.70,673019.51,...,1884568.96,1084.06,322.94,228070688.0,0.0,51446960.9,0.0,6433.35,10506.59,1394383.48
3.238960e-08,0.0,0.0,439796.86,1551523.68,0.01,40422679.9,1.66,20644.00,141394.72,669205.40,...,1870290.94,1084.08,240.41,227516629.0,0.0,50584906.3,0.0,6433.22,10506.58,1235096.61
3.310664e-08,0.0,0.0,443847.06,1551949.04,0.01,39392628.8,1.66,17471.62,141392.70,665329.19,...,1855808.44,1084.10,177.81,226951645.0,0.0,49718680.4,0.0,6433.10,10506.58,1091071.91
3.383955e-08,0.0,0.0,447782.55,1552191.70,0.00,38361178.8,1.66,14732.19,141390.63,661390.38,...,1841121.00,1084.11,130.64,226375559.0,0.0,48848591.8,0.0,6432.97,10506.58,961199.96
3.458869e-08,0.0,0.0,451599.08,1552264.34,0.00,37329151.5,1.67,12375.44,141388.51,657388.46,...,1826228.34,1084.13,95.33,225788192.0,0.0,47974961.3,0.0,6432.84,10506.57,844414.44
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9.180324e+01,0.0,0.0,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.00,0.00,0.00
9.383557e+01,0.0,0.0,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.00,0.00,0.00
9.591290e+01,0.0,0.0,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.00,0.00,0.00


The time is in years and the power densitites are in W/g.

The time is the specified time at which we want to sort the power densities. 

If time is not specified, it will be assumed that power_densities has only 1 column.
power_densities_df is the dataframe of power densities. It must be specified. 

time_array is the array of times. If not specified, it will be assumed that the time_array
is the index of the power_densities_df.

min_index is the index of the time_array at which we want to start sorting. If not specified,
it will be found using the function named binary_search.

max is a boolean that indicates if we want the smallest time larger than the requested value.

In [20]:
def binary_search(arr, low, high, x):
 
    # Check base case
    if high >= low:
 
        mid = int((high + low) / 2)
 
        # If element is present at the middle itself
        if arr[mid] == x:
            return mid
 
        # If element is smaller than mid, then it can only
        # be present in left subarray
        elif arr[mid] > x:
            return binary_search(arr, low, mid - 1, x)
 
        # Else the element can only be present in right subarray
        else:
            return binary_search(arr, mid + 1, high, x)
 
    else:
        # Element is not present in the array
        return high, low

def sort_power_densities(power_densities_df, time = -1, time_array  = [], min_index = [],
                            want_max = True):
    '''
    The time is in years and the power densitites are in W/g.

    The time is the specified time at which we want to sort the power densities. 

    If the time is not specified, it will be assumed that power_densities is has only 1 column.

    power_densities_df is the dataframe of power densities. It must be specified. 

    time_array is the array of times. If not specified, it will be assumed that the time_array
    
    is the index of the power_densities_df.

    min_index is the index of the time_array at which we want to start sorting. If not specified,
    
    it will be found using the function named binary_search.

    want_max is a boolean that indicates if we want the smallest time larger than the requested value.
    '''
    if(type(min_index) is list):
        if(time == -1):
            return power_densities_df.sort_values(ascending = False)
        if(len(time_array) == 0):
            time_array = power_densities_df.index.values            
        if(want_max):
            min_index = max(binary_search(time_array, 0, len(time_array)-1, time))
        else:
            min_index = min(binary_search(time_array, 0, len(time_array)-1, time))
    return power_densities_df.iloc[min_index,:].sort_values(ascending = False)

In [21]:
maximum_power_densities = {}
maximum_power_densities['Power Density (W/g)'] = [sort_power_densities(power_densities_df,
                         min_index = row)[0] for row in range(power_densities_df.shape[0])]
maximum_power_densities['Isotope'] = [sort_power_densities(power_densities_df,
                         min_index = row).index[0] for row in range(power_densities_df.shape[0])]
maximum_power_densities['Time (years)'] = power_densities_df.index
#maximum_power_densities['Decay Chain'] = [make_decay_chain(isotope, 
#    isotope_list, lambda_list, decay_energy_list, daughter_list) for isotope in maximum_power_densities['Isotope']]
maximum_power_densities_df = pd.DataFrame(maximum_power_densities)
maximum_power_densities_df

,Power Density (W/g),Isotope,Time (years)
0,2.213397e+09,205Ac,3.168809e-08
1,2.157775e+09,205Ac,3.238960e-08
2,2.103543e+09,200At,3.310664e-08
3,2.077118e+09,200At,3.383955e-08
4,2.050341e+09,200At,3.458869e-08
...,...,...,...
995,4.800000e-01,57K,9.180324e+01
996,4.600000e-01,57K,9.383557e+01
997,4.400000e-01,57K,9.591290e+01
998,4.200000e-01,57K,9.803621e+01


In [22]:
def plot_maximum_power_densities(maximum_power_densities_df, time = 0, file_name = -1):
    if(time != 0):
        min_row = max(binary_search(maximum_power_densities_df['Time (years)'], 0,
                            maximum_power_densities_df.shape[0]-1, time))
        maximum_power_densities_df = maximum_power_densities_df.iloc[min_row:,:]
    fig = px.scatter(maximum_power_densities_df, x = 'Time (years)', y = 'Power Density (W/g)',
                    hover_name= 'Isotope', log_x = True, log_y = True, 
                    color = 'Isotope', color_continuous_scale = 'algae')
    fig.show()
    if type(file_name) is not int:
        fig.write_image(file_name)
    
plot_maximum_power_densities(maximum_power_densities_df, .1, 
    file_name = subprocess.os.getcwd() + '\\maximum_power_densities_0.1_years.PDF')
#export figure as PDF


Now for some rough order of magnitude to find the total energy from each parent nucleus.

In [15]:
def return_energy(atomic_mass, min_power_density, min_life_time):
    '''
    This function returns the energy of the most energetic isotope of a given mass.
    Expected units are (none, W, years)
    '''
    decays = constants.N_A * units.mol / atomic_mass 
    total_energy = min_power_density * 12 * units.year.to(units.second) * units.J.to(units.eV)
    return total_energy / decays
np.log10(return_energy(228, 3.9, 12))

<Quantity 6.54282337>

In [10]:
max_isotopes = ['253Cf', '254Cf', '254Es', '228Th', '228Ra', '241Pu', '207Bi']
max_isotopes_decay_chains = {}
for isotope in max_isotopes:
    max_isotopes_decay_chains[isotope] = make_decay_chain(isotope, 
    isotope_list, lambda_list, decay_energy_list, daughter_list)
max_isotopes_decay_chains

{'253Cf':       e-Folding Time (seconds) Average beta-decay energy Daughter
 253Cf                 2.22e+06                      5921    253Es
 253Es              2.55156e+06                      6166    253Fm
 253Fm                   373947                      6486    253Md
 253Md                  1038.74                      5.69    253No
 253No                  135.902                      5.69    253Lr
 253Lr                 0.911783                      5.69    253Rf
 253Rf                 0.018755                      5.69    253Db,
 '254Cf':       e-Folding Time (seconds) Average beta-decay energy Daughter
 254Cf              7.54126e+06                      5.69    254Es
 254Es              3.43657e+07                      6357    254Fm
 254Fm                  16827.6                      7050    254Md
 254Md                  865.617                      5.69    254No
 254No                   73.866                      5.69    254Lr
 254Lr                  17.3123            

In [43]:


def mean_power_density(power_densities, time_array = None, min_time = None, max_time = None,
                        inclusive = True, sort = None):
    '''
    This function takes a time series of power densities and returns the average
    power density (which can be a numpy array or dataframe). 

    It only takes one time series (one decay chain) and returns the mean.

    The indices are the time for each point of the time series. 

    Inclusive is a boolean that indicates whether or not there the nearest time point outside of 
    the time interval is included in the average.
    '''
    try:
        if (min_time):
            min_index = binary_search(time_array, 0, 
                        len(time_array)-1, min_time)
            if(inclusive and len(min_index) > 1):
                min_index = min(min_index)
        else:
            min_index = 0
        if (max_time):
            max_index = binary_search(time_array, 0, 
                        len(time_array)-1, max_time)
            if(inclusive and len(max_index) > 1):
                max_index = max(max_index)
        else:
            max_index = -1
        if(min_index == max_index): #only one point in the desired range
            return (power_densities[min_index], time_array[min_index])
        try:
            if len(sort) > 0 :
                output = {}
                output['Power Densities'] = np.mean(power_densities[min_index:max_index])
                output['Time'] = time_array[min_index:max_index]
                return pd.DataFrame(output).sort_index(by = sort, inplace = False)
            if(max_index < 0):
                return (np.mean(power_densities[min_index:]), time_array[min_index])
            return (np.mean(power_densities[min_index:max_index]), time_array[min_index])
        except:
            return np.mean(power_densities[min_index:max_index], time_array[min_index:max_index])
    except:
        return np.mean(power_densities)

#Loop through columns in power density dataframe




In [22]:
oneY_to_3Y_power_densities = [mean_power_density(power_densities_df[column], 
                                time_array, min_time = 1 * units.year.to(units.second), 
                                max_time = 3 * units.year.to(units.second))
                                for column in power_densities_df.columns]
oneY_to_3Y_power_densities

NameError: name 'mean_power_density' is not defined

In [46]:
a = np.array(10 * [10 * [1]])
a.shape, type(a) == np.ndarray, type(power_densities_df) == pd.core.frame.DataFrame, 


((10, 10),
 True,
 True,
 array([3.16880878e-08, 3.23895968e-08, 3.31066359e-08, 3.38395486e-08,
        3.45886866e-08, 3.53544089e-08, 3.61370827e-08, 3.69370834e-08,
        3.77547943e-08, 3.85906078e-08, 3.94449244e-08, 4.03181538e-08,
        4.12107148e-08, 4.21230352e-08, 4.30555524e-08, 4.40087138e-08,
        4.49829761e-08, 4.59788066e-08, 4.69966827e-08, 4.80370925e-08,
        4.91005349e-08, 5.01875196e-08, 5.12985680e-08, 5.24342127e-08,
        5.35949983e-08, 5.47814812e-08, 5.59942304e-08, 5.72338274e-08,
        5.85008666e-08, 5.97959553e-08, 6.11197147e-08, 6.24727794e-08,
        6.38557981e-08, 6.52694340e-08, 6.67143649e-08, 6.81912836e-08,
        6.97008983e-08, 7.12439327e-08, 7.28211267e-08, 7.44332366e-08,
        7.60810352e-08, 7.77653127e-08, 7.94868767e-08, 8.12465525e-08,
        8.30451839e-08, 8.48836334e-08, 8.67627823e-08, 8.86835317e-08,
        9.06468025e-08, 9.26535361e-08, 9.47046947e-08, 9.68012617e-08,
        9.89442424e-08, 1.01134664e-07,

True